In [1]:
# @title Install Necessary Packages
!pip install autotrain-advanced
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 42.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.8 MB/s eta 

In [2]:
!autotrain setup --update-torch

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from peft import PeftModel
import torch

from huggingface_hub import snapshot_download

Downloading only checkpoint folder from my HF repo, Not downloading weights from there because of its size 25GB. Instead we use Llama2 sharded version as base model and the adapter file from checkpoint

In [4]:
snapshot_download(repo_id="Sriram-Gov/Sarcastic-Headline-Llama2", local_dir = '/content', allow_patterns="checkpoint-672/*") # , local_dir = '/content/exp', revision="refs/pr/1"


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

'/content'

We are taking sharded Llama2 7B sharded version as base model here, because it is compressed of size 13GB instead of 25GB.<br>
However, if you want to download the 25GB [version](https://huggingface.co/Sriram-Gov/Sarcastic-Headline-Llama2) available in HF repo, you can also do inference in that way

In [6]:
tokenizer = AutoTokenizer.from_pretrained('/content/checkpoint-672')
model = AutoModelForCausalLM.from_pretrained('TinyPixel/Llama-2-7B-bf16-sharded', torch_dtype = torch.float16, device_map="auto") #Base_Model for example: meta-llama/Llama-2-13b-chat-hf
model = PeftModel.from_pretrained(model, "/content/checkpoint-672", device_map="auto")

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [7]:
# @title Prompt template
format_text = "You are a savage, disrespectful and witty agent. You convert below news headline into a funny, humiliating, creatively sarcastic news headline while still maintaining the original context.\n### headline: {}\n### sarcastic_headline: {}"
print(format_text)

You are a savage, disrespectful and witty agent. You convert below news headline into a funny, humiliating, creatively sarcastic news headline while still maintaining the original context.
### headline: {}
### sarcastic_headline: {}


In [11]:
t1 = "a couple sitting on a desk having the time of their life"
t2 = "steriods are good for lungs"
t3 = 'mansoons are best for mosquitoes'
t4 = 'i love to eat pizza'
formatted_input = format_text.format(t2, "")
print(formatted_input)

You are a savage, disrespectful and witty agent. You convert below news headline into a funny, humiliating, creatively sarcastic news headline while still maintaining the original context.
### headline: steriods are good for lungs
### sarcastic_headline: 


In [12]:
device = "cuda:0"

inputs = tokenizer(formatted_input, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length=300) # max_length=300, temperature=1
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

You are a savage, disrespectful and witty agent. You convert below news headline into a funny, humiliating, creatively sarcastic news headline while still maintaining the original context.
### headline: steriods are good for lungs
### sarcastic_headline:  steroids are good for lungs, because they make you breathe more heavily and deeply, which is just like exercise
